In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torchvision.models import resnet18, ResNet18_Weights

In [2]:
# 모델 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [3]:
import time
start_time = time.time()

# 코드 중간중간
print("학습 시작 전 체크포인트 통과")
...
print("Epoch 1 시작")

# 이후
print(f"총 준비 시간: {time.time() - start_time:.2f}초")

학습 시작 전 체크포인트 통과
Epoch 1 시작
총 준비 시간: 0.00초


In [4]:
# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),                # ResNet은 224x224 입력
    transforms.Grayscale(num_output_channels=3),  # 1채널 -> 3채널
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)  # 정규화 (모델 성능 안정화)
])


In [5]:
# 데이터 로딩
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [6]:
# 모델 로딩 및 설정
weights = ResNet18_Weights.DEFAULT
model = resnet18(weights=weights)
model.fc = nn.Linear(model.fc.in_features, 10)
model = model.to(device)

In [12]:
# 손실 함수와 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [ ]:
# 학습 루프
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

In [ ]:
# 테스트
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

print(f"\nTest Accuracy: {100 * correct / total:.2f}%")

In [ ]:
print(torchvision.__version__